# Physical-Mathematical Model : BP = f(VCG)

In [3]:
#@title Connect to gDrive & Load Files
ENV_COLAB = 'google.colab' in str(get_ipython())
if ENV_COLAB:
    print('Running on CoLab')
    PROJECT_DRIVE_FOLDER= "/content/drive/MyDrive/NiPAMS/BP development/analysis/nipams-bp-py-main/" #@param {"type":"string"}
    ## Load this for local development version of library.. Loads from GoogleDrive (changes persist)
    from google.colab import drive, widgets, data_table; 
    drive.mount('/content/drive');
    %cd "$PROJECT_DRIVE_FOLDER"
  
    !pip install -qr requirements.txt
    !pip install -q envyaml
    import os, sys; 
    from envyaml import EnvYAML; env = EnvYAML('env/default.yml'); 
    for k,v in env.export().items(): os.environ[k]=str(v)
    sys.path.append(os.path.abspath("./src"))

else:
    print('Not running on CoLab')
    %cd /app
!pwd

from importlib import reload;
from utils.constants import *

Running on CoLab
Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1MnayBGJfAervEVUyYNEibuPedW7NJuzm/NiPAMS/BP development/analysis/nipams-bp-py-main
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.0/292.0 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.7/219.7 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.7 MB/s eta 0:00:00
/content/drive/.shortcut-targets-by-id/1MnayBGJfAervEVUyYNEibuPedW7NJuzm/

# 1. Load & Process Data

In [4]:
from data import load_data; reload(load_data)
from data import preprocess;

if REPROCESS:
  _dfBp, _dfImu = load_data.load_dataframe_from_mat(folder='data/raw_mat/', pattern=MAT_PATTERN, limit_files=LIMIT)

  dfAll = preprocess.merge_imu_vcg_with_heartbeats(_dfBp, _dfImu)
  dfAll[IMU_DATA_COLS] = dfAll[IMU_DATA_COLS].astype(np.float32)
  dfAll[BP_COLS] = dfAll[BP_COLS].astype(np.float32)
  dfAll.to_pickle(path=FILE_PROCESSED)

NameError: ignored

In [ ]:
# dfBp, dfImu = load_data.load_dataframe_from_mat(folder='data/raw_mat/', pattern='LLV', limit_files=30)
dfAll = load_data.load_dataframe_from_pickle(path=FILE_PROCESSED)
dfImu = dfAll[INDICIES + INFO_COLS + IMU_COLS]
dfBp = dfAll[INDICIES + INFO_COLS + BP_COLS]

display('------ dfAll ------')
display("Unique Files : ", dfAll.file.nunique())
display(np.unique(dfAll.file))
display(dfAll)

# Connect to GCS

In [1]:
import os
from utils.constants import *
%cd {os.environ['PROJECT_PATH']}
# os.environ['PYTHONPATH'] = os.environ['PYTHONPATH'] + '/src/'

ModuleNotFoundError: ignored

In [ ]:
# import os
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'creds/nipams-83df6260e5a8.json'
# !echo $GOOGLE_APPLICATION_CREDENTIALS

# Instantiates a client
from google.cloud import storage
client = storage.Client()

# import re
# re.match('asdf', 'data_cleaned_LVET_m/sub019_LLV1.mat')

bucket = client.get_bucket('data_cleaned_lvet')

# # https://stackoverflow.com/questions/22398898/google-cloud-storage-python-any-way-to-list-obj-in-certain-folder-in-gcs
blobs = list(bucket.list_blobs(10, prefix='data_cleaned_LVET_m/'))

In [ ]:
def getBpDataFromMatlab(dictImuData):
  bp_all = np.asarray(dictImuData['mlts_now'])
  dfBp = pd.DataFrame(bp_all, columns=BP_COLS)
  dfBp.index.name = 'heartbeat'
  return dfBp

def getImuDataFromMatlab(dictImuData):
  imu_all = np.asarray(dictImuData['mlfs_now'])
  imuShape = imu_all.shape
  imuInd = pd.MultiIndex.from_product([range(imuShape[2]), range(imuShape[0])], names=['heartbeat','step_index'])
  dfRaw = pd.concat([pd.DataFrame(imu_all[:,:,i]) for i in range(imuShape[2])]).reset_index(drop=True)
  dfRaw.columns = IMU_COLS
  dfRaw = dfRaw.set_index(imuInd).reset_index().drop('step_index', axis=1).set_index(['heartbeat','ts'])
  return dfRaw

def getContinuousDataFromMatlab(dictImuData):
  dfBpRaw = pd.concat([
    pd.DataFrame(dictImuData['bio_data'][:,[17,14,15]], columns=BP_COLS),
    pd.DataFrame(dictImuData['bio_ts']).T.rename(columns={0:'ts'}),
    pd.DataFrame(dictImuData['bio_data'][:,12], columns=['ecgTs'])
  ], axis=1)
  dfBpRaw.ts = pd.to_timedelta(dfBpRaw.ts, unit='s')

  dfImuRaw = pd.concat([
    pd.DataFrame(dictImuData['imu_data'], columns=IMU_DATA_COLS),
    pd.DataFrame(dictImuData['imu_ts']).rename(columns={0:'ts'})
  ], axis=1)
  dfImuRaw.ts = pd.to_timedelta(dfImuRaw.ts, unit='s')

  return dfImuRaw.set_index('ts'), dfBpRaw.set_index('ts')


In [ ]:
folder=''
pattern='.*\.mat$'
continuous=True

matchedBlobs = [blob for blob in blobs if re.match(pattern, blob.name)][:10]

blobFileNames = {}
arrDfsImu, arrDfsBp = [], []
for blob in matchedBlobs:
    blob.download_to_filename('data/raw_mat/'+blob.name.split('/')[-1])
    imuFile = loadmat('data/raw_mat/'+blob.name.split('/')[-1])

    if True:
        dfImu, dfBp = getContinuousDataFromMatlab(imuFile)
    else:
        dfBp = getBpDataFromMatlab(imuFile)
        dfImu = getImuDataFromMatlab(imuFile)
    dfBp['file'] = blob.name
    dfImu['file'] = blob.name
    arrDfsBp.append(dfBp)
    arrDfsImu.append(dfImu)

In [ ]:
blob = blobs[2]
blob.download_to_filename('data/raw_mat/'+blob.name.split('/')[-1])
imuFile = loadmat('data/raw_mat/'+blob.name.split('/')[-1])
  
dfImu, dfBp = getContinuousDataFromMatlab(imuFile)
# dfBp = getBpDataFromMatlab(imuFile)
# dfImu = getImuDataFromMatlab(imuFile)
dfBp['file'] = blob.name
dfImu['file'] = blob.name
arrDfsBp.append(dfBp)
arrDfsImu.append(dfImu)

In [ ]:
dfBp.pp.value_counts(dropna=False)

In [ ]:
from data import load_data
from importlib import reload
reload(load_data)

In [ ]:
load_data.connect_gcs()
dfImuAll, dfBpAll = load_data.load_data_gcs()


In [ ]:
dfImuAll, dfBpAll = load_data.load_dataframe_from_mat('data/raw_mat/')
dfImuCont, dfBpCont = dfImuAll, dfBpAll


# 1) Load & Prep Nipams Data

In [ ]:
import os
from utils.constants import *
%cd {os.environ['PROJECT_PATH']}

In [ ]:
from data import load_data
from importlib import reload
reload(load_data)

In [ ]:
dfImuAll, dfBpAll = load_data.load_dataframe_from_mat('data/raw_mat/')
dfImuCont, dfBpCont = dfImuAll, dfBpAll
dfImuCont.file.value_counts()

# 2) Preprocess

In [ ]:
import nipams_ml as nm
# reload(nm.preprocess)

## 2.1) Prepocess - James

In [2]:
dfBp, dfImuRaw = nm.preprocess.filterSplit(dfBpCont.set_index('ts'),dfImuCont.set_index('ts'))
dfImuRaw['ts'] = dfImuRaw.index.total_seconds()
dfImuRaw = dfImuRaw.reset_index(drop=True)
dfImuRaw = dfImuRaw.set_index('file')

dfAll = dfImuRaw.merge(dfBp, how='left', on=nm.constants.INDICIES)
dfAll.ts = pd.to_timedelta(dfAll.ts*1e9)

NameError: ignored

# 3) Define the Model

In [ ]:
#@title Helper Methods

########################## DATA HELPERS ##########################
## Return the IMU & BP data for a specific heartbeat
def get_vcg_data_for_hb(selectedFile, selectedHb, dfAll=dfAll):
  # Get data for individual heartbeat
  selected_data = dfAll[(dfAll.file.str.contains(selectedFile)) &\
                                  (dfAll['heartbeat'] == selectedHb)]
  selected_data = selected_data.set_index('ts')
  # selected_data = selected_data.fillna(method='ffill').fillna(method='bfill')
  return selected_data


########################## MATH HELPERS ##########################
## Return the Double Integral of Signal (aka Vector)
from scipy import integrate
def displacement(vcg_sig):
  vcg_sig = vcg_sig - vcg_sig.mean()   # TODO : Remove 'accelerometer drift'?

  # Double Integral
  serX=vcg_sig.index.astype(int)/1e9
  vz = integrate.cumtrapz(y=vcg_sig,x=serX)-vcg_sig.mean()   ## TODO : is this the best approach?
  # serVzX=vz.index.astype(int)/1e9
  dz = integrate.cumtrapz(y=vz-vz.mean(), x=serX[1:])
  return dz, vz


## Calculate RMS Envelope -- TODO : use timestamp for pd.rolling instead of np.convolve
def window_rms(a, window_size):
  a2 = np.power(a,2)
  window = np.ones(window_size)/float(window_size)
  return np.sqrt(np.convolve(a2, window, 'valid'))


########################## Plugin HELPERS ##########################
## Plugin function to allow 'ingecting' code snippets during execution
def plugin(name, pluginFns, localDict):
  fns = pluginFns.get(name, [])
  fns = [fns] if isinstance(fns, str) else fns
  [exec(fn['fn'], globals(), {**localDict, **fn['context']}) if isinstance(fn, dict)
      else exec(fn, globals(), localDict) 
      for fn in fns]
## Take snapshots of important data while executing..
def DataLog():
  db = {}
  def save(name, data):
    if name not in db : db[name] = []
    db[name].append(data)
  return db, save

![](https://drive.google.com/uc?export=view&id=1ui2A_fUNQqOJ3uxgx_yfMOpv_cMXDvj5)

Image Source : https://drive.google.com/file/d/1ui2A_fUNQqOJ3uxgx_yfMOpv_cMXDvj5/view?usp=sharing


In [ ]:
#@title 🧠 Implement Physical-Mathematical Model

## Input : Data-VCG for single heartbeat. Output : Calculated BP
def calculate_BP_from_VCG(df_vcg, params={'k1':1.,'k2':1.}, pluginFns={}):
  plugin('start',pluginFns, locals())
  
  ##### Get Vectors (signals) from df_vcg #####
  vcg_xp = df_vcg['az']
  # vcg_aar = df_vcg['ax']    # TODO : Fetch VCG_Aar, using ax for now..
  plugin('vcg',pluginFns, locals())

  ##### Acceleration -> Displacement #####
  dXp, vXp = displacement(vcg_xp)
  # dXp_y, vXp_y = displacement(df_vcg['ay'])
  dXp_z, vXp_z = displacement(df_vcg['az'])
  # dXp_Norm = np.linalg.norm(np.array([dXp,dXp_y,dXp_z]), axis=0)
  dXp_Norm = np.linalg.norm(np.array([dXp,dXp_z]), axis=0)
  
  # dAar, vAar = displacement(vcg_aar)
  # deltaDisp = dXp - dAar
  deltaDisp = dXp_Norm
  plugin('disp',pluginFns, locals())

  ##### Respiration Volume #####
  RV = window_rms(vcg_xp, 20).max()
  plugin('RV',pluginFns, locals())

  ##### BP at Aorta #####
  bp_Aar = params['k1'] * deltaDisp / RV    ## TODO : is trimming this to size okay?

  ##### Beat to Beat #####
  BTB = (vcg_xp.index.max() - vcg_xp.index.min()).total_seconds()

  BP_f = BTB * bp_Aar + params['k2']
  
  ##### Finalize SBP and DBP #####
  SBP = BP_f.max()
  DBP = BP_f.min()
  plugin('BP',pluginFns, locals())
  
  ##### End #####
  plugin('end',pluginFns, locals())
  return SBP, DBP


In [ ]:
#@title 🔎  Select Individual File-Heartbeat and Calculate

##### Choose Specific File & HeartBeat -OR- Random.. #####
# selectedHb, selectedFile = 3.0, 'sub200/sub200_HLV1.mat'
selectedFile, selectedHb = dfAll[['file','heartbeat']].sample(1).iloc[0].to_list()   ## Choose Random from DF

## Filter for VCG Data for Specific Heartbeat
dfHb = get_vcg_data_for_hb(selectedFile, selectedHb)

## Pass VCG Data and Calculate BP
SBP, DBP = calculate_BP_from_VCG(dfHb)
print(f'SBP: {SBP.round(2)} -- DBP: {DBP.round(2)}\n')

# 4) Run & Evaluate Model

In [ ]:
#@title 🏃 Run Model For All Heartbeats
########################## SETUP TEST ##########################
NUM_HEARTBEATS = 30 #@param
if NUM_HEARTBEATS == -1:
  dfAll_ = dfAll.copy()
else : 
  randomHeartBeats = dfAll.groupby(['file','heartbeat']).count().sample(NUM_HEARTBEATS).index
  dfAll_ = dfAll.set_index(['file','heartbeat']).loc[randomHeartBeats].reset_index()

########################## LOGGERS (Optional) ##########################
## Define Logging functions (optional) to be injected in calculation steps
dataLogs, save = DataLog()
pluginFns={
    'disp':['save("az",vcg_xp.values)'],
    'RV':['save("vz",vXp)','save("dz",deltaDisp)'],
    'BP':['save("BTB",BTB)','save("bp_Aar",bp_Aar)']
}

########################## ITERATE HEARTBEATS ##########################
## Iterate File-Heartbeats - Calculate BP for each
results = {}
for inds, dfHb in dfAll_.groupby(['file','heartbeat']):
  ## Fetch VCG IMU data for individual heartbeat (aka 'inds')
  dfHb = get_vcg_data_for_hb(*inds)

  ## Choose Params for Patient
  params = {'k1':1e5,'k2':60.}

  ## Calculate BP from VCG
  SBP, DBP = calculate_BP_from_VCG(dfHb, params, pluginFns)
  
  ## Save Results
  result = {'sbp_pred':SBP,'dbp_pred':DBP,
                   'sbp':dfHb['sbp'].mean(),'dbp':dfHb['dbp'].mean()
  }
  results[inds] = result
  print(inds, dfHb.shape, ' ---> ', result)

dfResults = pd.DataFrame.from_dict(results).T

# 5) 🎉 Review Results & Evaluate

In [ ]:
#@title Visualize VCG Signals (az, vz, dz)
# import ipywidgets as widgets

arrSigs = list(dataLogs.keys())
# arrSigs = list(dataLogs.keys())
# t = widgets.TabBar(arrSigs)
for i in arrSigs:
#   with t.output_to(i,select=False):
    [pd.Series(ser).plot(figsize=(16,8), label=dfResults.iloc[ind].dbp, legend=True) for ind,ser in enumerate(dataLogs[i])]

In [ ]:
#@title Evaluate with MAE, R2 & Correlation Plots
import sklearn.metrics as skm

arrBps = ['sbp','dbp']
# t = widgets.TabBar(arrBps)
for i in arrBps:
  # with t.output_to(i,select=False):
    yTarget, yPreds = dfResults[i], dfResults[f'{i}_pred']

    objScores = {
      'RMSE': skm.mean_squared_error(yTarget, yPreds, squared=False),
      'MSE': skm.mean_squared_error(yTarget, yPreds, squared=True),
      'mean_absolute_error': skm.mean_absolute_error(yTarget, yPreds),
      'STD of Absolute Error': np.abs(yTarget - yPreds).std(),
      'STD of Error': (yTarget - yPreds).std(),
      'r2_score': skm.r2_score(yTarget, yPreds)
    }
    display('Evaluation Metrics', pd.DataFrame([objScores]).T)

    display('Correlation : ', dfResults[[i,f'{i}_pred']].corr())
    dfResults.plot.scatter(i,f'{i}_pred')

In [ ]:
STOP_HERE

##

# 6) Build Capatible Estimator

In [ ]:
# https://stackoverflow.com/questions/31259891/put-customized-functions-in-sklearn-pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
class RandomRegressor():
  def fit(self, X, Y):
    self.serY = pd.Series(Y)
    return self
  def predict(self, rows, *args):
    return self.serY.sample(len(rows), replace=True).values
  def score(self, rows, y):
    return np.abs((self.predict(rows))-y)

def printer(x):
  print(x)
  return x

printerTrans = FunctionTransformer(printer)

In [ ]:
dfImu = dfAll[nm.constants.INDICIES + nm.constants.IMU_COLS]
dfBp = dfAll[nm.constants.INDICIES + nm.constants.BP_COLS]
sampleRandTestInds = nm.experiments.split_by_random(dfImu, dfBp)
sampleExpDfs = nm.experiments.get_experiment(sampleRandTestInds[0], dfImu, dfBp)
# nm.experiments.split_by_random(dfAll[nm.constants.IMU_COLS])

In [ ]:
RandomRegressor().fit(sampleExpDfs['test_x'],sampleExpDfs['test_y']['dbp']).predict(sampleExpDfs['test_y']['dbp'])

In [ ]:
def forceShape(x, nrows=300):
  x = x[:nrows]
  return np.pad(x, ((0,nrows-x.shape[0]),(0,0)))

# arrData = np.array(list(dfAll.groupby(nm.constants.INDICIES).apply(lambda x : forceShape(x[['az','ax']].values))))

def explode_3d(input_series, indicies=nm.constants.INDICIES, data_cols=nm.constants.IMU_DATA_COLS):
  '''
    Converts timeseries signals dataframe into 3d matrix of ts samples
  '''
  # dfTsVect = input_series[indicies + data_cols].groupby(indicies, sort=False).fillna(0)
  arrTsExp = np.array(list(
      input_series\
        .groupby(nm.constants.INDICIES)\
        .apply(lambda x : forceShape(x[['ts','az','ax']].values).T)
      )
  )
  return arrTsExp


In [ ]:
def vectorize_mean(input_series, indicies=nm.constants.INDICIES, data_cols=nm.constants.IMU_DATA_COLS):
  dfTsVect = input_series[indicies + data_cols].groupby(indicies, sort=False).agg(['mean','std']).fillna(0)
  dfTsVect.columns = ['_'.join(col).strip() for col in dfTsVect.columns.values]
  return dfTsVect

tsVectTransform = FunctionTransformer(vectorize_mean)

pipe = Pipeline([('scaler', tsVectTransform), ('rand', RandomRegressor())])
pipe.fit(sampleExpDfs['train_x'], sampleExpDfs['train_y'].groupby(nm.constants.INDICIES)['dbp'].mean())
pipe.score(sampleExpDfs['test_x'], sampleExpDfs['test_y'].groupby(nm.constants.INDICIES)['dbp'].mean())
preds = pipe.predict(sampleExpDfs['test_x'])
preds

In [ ]:
tsExplodeTransform = FunctionTransformer(explode_3d)

pipe = Pipeline([('scaler', tsExplodeTransform), ('rand', RandomRegressor())])
pipe.fit(sampleExpDfs['train_x'], sampleExpDfs['train_y'].groupby(nm.constants.INDICIES)['dbp'].mean())
pipe.score(sampleExpDfs['test_x'], sampleExpDfs['test_y'].groupby(nm.constants.INDICIES)['dbp'].mean())
preds = pipe.predict(sampleExpDfs['test_x'])
preds

In [ ]:
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

# https://scikit-learn.org/stable/developers/develop.html
class AnalyticalBPEstimator():
  def __init__(self, k1=700.,k2=100.):
    self.k1 = k1
    self.k2 = k2

  def fit(self, X, y):
    ## GridSearch Params
    
    # self.X_ = X
    # self.y_ = y
    # Return the classifier
    return self

  def predict(self, X):

    # closest = np.argmin(euclidean_distances(X, self.X_), axis=1)
    y_preds = []
    for row in X:
      dfHeartBeat = pd.DataFrame(row.T, columns=['ts','az','ax']).replace(0, np.NaN).set_index('ts').dropna()
      y_pred = calculate_BP_from_VCG(dfHeartBeat[dfHeartBeat.index!=0], params={'k1':self.k1, 'k2':self.k2})
      y_preds.append(y_pred)
    return y_preds

In [ ]:
X = tsExplodeTransform.fit_transform(sampleExpDfs['train_x'])

In [ ]:
# X[0], sampleExpDfs['train_y'][0]
# sampleExpDfs['train_y']['sbp'].iloc[0]

In [ ]:
modAnalytical = AnalyticalBPEstimator().fit(X, sampleExpDfs['train_y'])
modAnalytical.predict(X)

In [ ]:
#@title Grid Search Model Parameters

heartBeatNum = 0
arrK1 = [1e1,1e2,1e3,1e4,1e5]
arrK2 = [1e1,1e2,1e3,1e4,1e5]

print(sampleExpDfs['train_y']['sbp'].iloc[0])
for k1 in arrK1:
  for k2 in arrK2:
    modAnalytical.k1 = k1
    modAnalytical.k2 = k2
    preds = modAnalytical.predict(X[:1])
    print(k1, k2, preds)

# ROUGH

In [ ]:
STOP_HERE